In [3]:
# Import des librairies nécessaires
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint

# Montage de Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Chemin d'accès au dataset
base_dir = '/content/drive/My Drive/vietnamdataset/Dataset'

# Préparation des données
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2)  # Utilisation de 20% des images pour la validation

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

# Construction du modèle
# model = Sequential([
#     Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
#     MaxPooling2D(2, 2),
#     Conv2D(64, (3, 3), activation='relu'),
#     MaxPooling2D(2, 2),
#     Conv2D(128, (3, 3), activation='relu'),
#     MaxPooling2D(2, 2),
#     Flatten(),
#     Dense(512, activation='relu'),
#     Dropout(0.5),
#     Dense(len(train_generator.class_indices), activation='softmax')
# ])

#alexNet

model = Sequential([
    # Première couche convolutive avec l'input shape
    Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((3, 3), strides=(2, 2)),

    # Deuxième couche convolutive
    Conv2D(256, (5, 5), padding='same', activation='relu'),
    MaxPooling2D((3, 3), strides=(2, 2)),

    # Troisième couche convolutive
    Conv2D(384, (3, 3), padding='same', activation='relu'),

    # Quatrième couche convolutive
    Conv2D(384, (3, 3), padding='same', activation='relu'),

    # Cinquième couche convolutive
    Conv2D(256, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((3, 3), strides=(2, 2)),

    # Aplatir les sorties pour les couches denses
    Flatten(),

    # Couches denses
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(4096, activation='relu'),
    Dropout(0.5),

    # Couche de sortie
    Dense(len(train_generator.class_indices), activation='softmax')
])

#mobilenet

from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, BatchNormalization, ReLU, GlobalAveragePooling2D, Dense, Dropout



# model = Sequential([
#     # Input layer
#     Conv2D(32, (3, 3), strides=(2, 2), padding='same', activation='relu', input_shape=(224, 224, 3)),


#     # Depthwise Separable Convolution Block 1
#     DepthwiseConv2D((3, 3), padding='same'),

#     Conv2D(64, (1, 1), padding='same'),


#     # Depthwise Separable Convolution Block 2
#     DepthwiseConv2D((3, 3), strides=(2, 2), padding='same'),

#     Conv2D(128, (1, 1), padding='same'),


#     # Depthwise Separable Convolution Block 3
#     DepthwiseConv2D((3, 3), padding='same'),

#     Conv2D(128, (1, 1), padding='same'),


#     # Depthwise Separable Convolution Block 4
#     DepthwiseConv2D((3, 3), strides=(2, 2), padding='same'),

#     Conv2D(256, (1, 1), padding='same'),


#     # Global Average Pooling
#     GlobalAveragePooling2D(),

#     # Dense layers
#     Dense(256, activation='relu'),
#     Dropout(0.5),
#     Dense(len(train_generator.class_indices), activation='softmax')
# ])




#======================================================================




from tensorflow.keras.layers import DepthwiseConv2D, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, BatchNormalization, GlobalAveragePooling2D

# model = Sequential([
#     # Couche d'entrée
#     Conv2D(32, (3, 3), strides=(2, 2), padding='same', activation='relu', input_shape=(224, 224, 3)),
#     BatchNormalization(),

#     # Depthwise Convolution Blocks
#     DepthwiseConv2D((3, 3), padding='same', strides=(1, 1), depth_multiplier=1),
#     BatchNormalization(),
#     Activation('relu'),
#     Conv2D(64, (1, 1), padding='same', activation='relu'),
#     BatchNormalization(),

#     DepthwiseConv2D((3, 3), padding='same', strides=(2, 2), depth_multiplier=1),
#     BatchNormalization(),
#     Activation('relu'),
#     Conv2D(128, (1, 1), padding='same', activation='relu'),
#     BatchNormalization(),

#     DepthwiseConv2D((3, 3), padding='same', strides=(1, 1), depth_multiplier=1),
#     BatchNormalization(),
#     Activation('relu'),
#     Conv2D(128, (1, 1), padding='same', activation='relu'),
#     BatchNormalization(),

#     DepthwiseConv2D((3, 3), padding='same', strides=(2, 2), depth_multiplier=1),
#     BatchNormalization(),
#     Activation('relu'),
#     Conv2D(256, (1, 1), padding='same', activation='relu'),
#     BatchNormalization(),

#     DepthwiseConv2D((3, 3), padding='same', strides=(1, 1), depth_multiplier=1),
#     BatchNormalization(),
#     Activation('relu'),
#     Conv2D(256, (1, 1), padding='same', activation='relu'),
#     BatchNormalization(),

#     DepthwiseConv2D((3, 3), padding='same', strides=(2, 2), depth_multiplier=1),
#     BatchNormalization(),
#     Activation('relu'),
#     Conv2D(512, (1, 1), padding='same', activation='relu'),
#     BatchNormalization(),

#     # Répéter ce bloc 5 fois pour les caractéristiques intermédiaires
#     *[Sequential([
#         DepthwiseConv2D((3, 3), padding='same', strides=(1, 1), depth_multiplier=1),
#         BatchNormalization(),
#         Activation('relu'),
#         Conv2D(512, (1, 1), padding='same', activation='relu'),
#         BatchNormalization(),
#     ]) for _ in range(5)],

#     DepthwiseConv2D((3, 3), padding='same', strides=(2, 2), depth_multiplier=1),
#     BatchNormalization(),
#     Activation('relu'),
#     Conv2D(1024, (1, 1), padding='same', activation='relu'),
#     BatchNormalization(),

#     # Couche de sortie
#     GlobalAveragePooling2D(),
#     Dense(len(train_generator.class_indices), activation='softmax')
# ])

#======================================================================

# model = Sequential([
#     # Initial Convolution layer
#     Conv2D(32, kernel_size=(3, 3), strides=(2, 2), padding='same', use_bias=False, input_shape=(224, 224, 3)),
#     BatchNormalization(),
#     ReLU(6.),

#     # Block 1
#     DepthwiseConv2D(kernel_size=(3, 3), padding='same', use_bias=False),
#     BatchNormalization(),
#     ReLU(6.),
#     Conv2D(64, kernel_size=(1, 1), padding='same', use_bias=False),
#     BatchNormalization(),
#     ReLU(6.),

#     # Block 2
#     DepthwiseConv2D(kernel_size=(3, 3), strides=(2, 2), padding='same', use_bias=False),
#     BatchNormalization(),
#     ReLU(6.),
#     Conv2D(128, kernel_size=(1, 1), padding='same', use_bias=False),
#     BatchNormalization(),
#     ReLU(6.),

#     # Block 3
#     DepthwiseConv2D(kernel_size=(3, 3), padding='same', use_bias=False),
#     BatchNormalization(),
#     ReLU(6.),
#     Conv2D(128, kernel_size=(1, 1), padding='same', use_bias=False),
#     BatchNormalization(),
#     ReLU(6.),

#     # Block 4
#     DepthwiseConv2D(kernel_size=(3, 3), strides=(2, 2), padding='same', use_bias=False),
#     BatchNormalization(),
#     ReLU(6.),
#     Conv2D(256, kernel_size=(1, 1), padding='same', use_bias=False),
#     BatchNormalization(),
#     ReLU(6.),

#     # Block 5
#     DepthwiseConv2D(kernel_size=(3, 3), padding='same', use_bias=False),
#     BatchNormalization(),
#     ReLU(6.),
#     Conv2D(256, kernel_size=(1, 1), padding='same', use_bias=False),
#     BatchNormalization(),
#     ReLU(6.),

#     # Block 6
#     DepthwiseConv2D(kernel_size=(3, 3), strides=(2, 2), padding='same', use_bias=False),
#     BatchNormalization(),
#     ReLU(6.),
#     Conv2D(512, kernel_size=(1, 1), padding='same', use_bias=False),
#     BatchNormalization(),
#     ReLU(6.),

#     # Blocks 7-11 (5 times)
#     DepthwiseConv2D(kernel_size=(3, 3), padding='same', use_bias=False),
#     BatchNormalization(),
#     ReLU(6.),
#     Conv2D(512, kernel_size=(1, 1), padding='same', use_bias=False),
#     BatchNormalization(),
#     ReLU(6.),

#     DepthwiseConv2D(kernel_size=(3, 3), padding='same', use_bias=False),
#     BatchNormalization(),
#     ReLU(6.),
#     Conv2D(512, kernel_size=(1, 1), padding='same', use_bias=False),
#     BatchNormalization(),
#     ReLU(6.),

#     DepthwiseConv2D(kernel_size=(3, 3), padding='same', use_bias=False),
#     BatchNormalization(),
#     ReLU(6.),
#     Conv2D(512, kernel_size=(1, 1), padding='same', use_bias=False),
#     BatchNormalization(),
#     ReLU(6.),

#     DepthwiseConv2D(kernel_size=(3, 3), padding='same', use_bias=False),
#     BatchNormalization(),
#     ReLU(6.),
#     Conv2D(512, kernel_size=(1, 1), padding='same', use_bias=False),
#     BatchNormalization(),
#     ReLU(6.),

#     DepthwiseConv2D(kernel_size=(3, 3), padding='same', use_bias=False),
#     BatchNormalization(),
#     ReLU(6.),
#     Conv2D(512, kernel_size=(1, 1), padding='same', use_bias=False),
#     BatchNormalization(),
#     ReLU(6.),

#     # Block 12
#     DepthwiseConv2D(kernel_size=(3, 3), strides=(2, 2), padding='same', use_bias=False),
#     BatchNormalization(),
#     ReLU(6.),
#     Conv2D(1024, kernel_size=(1, 1), padding='same', use_bias=False),
#     BatchNormalization(),
#     ReLU(6.),

#     # Block 13
#     DepthwiseConv2D(kernel_size=(3, 3), padding='same', use_bias=False),
#     BatchNormalization(),
#     ReLU(6.),
#     Conv2D(1024, kernel_size=(1, 1), padding='same', use_bias=False),
#     BatchNormalization(),
#     ReLU(6.),

#     # Global Average Pooling and Fully connected layer
#     GlobalAveragePooling2D(),
#     Dense(13, activation='softmax')
# ])




model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])


checkpoint_filepath = 'model.keras'

# Créer un callback pour enregistrer le meilleur modèle basé sur la validation accuracy
checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)


# Entraînement du modèle
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[checkpoint])





Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 1019 images belonging to 13 classes.
Found 253 images belonging to 13 classes.
Epoch 1/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 464ms/step - accuracy: 0.0744 - loss: 2.5724
Epoch 1: val_accuracy improved from -inf to 0.08036, saving model to model.keras
31/31 ━━━━━━━━━━━━━━━━━━━━ 34s 806ms/step - accuracy: 0.0744 - loss: 2.5724 - val_accuracy: 0.0804 - val_loss: 2.5597
Epoch 2/100
 1/31 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.0625 - loss: 2.5600
Epoch 2: val_accuracy did not improve from 0.08036
31/31 ━━━━━━━━━━━━━━━━━━━━ 4s 148ms/step - accuracy: 0.0625 - loss: 2.5600 - val_accuracy: 0.0690 - val_loss: 2.5592
Epoch 3/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step - accuracy: 0.0851 - loss: 2.5523
Epoch 3: val_accuracy improved from 0.08036 to 0.19196, saving model to model.keras
31/31 ━━━━━━━━━━━━━━━━━━━━ 19s 512ms/step - accuracy: 0.0859 - loss: 2.5518